In [3]:
!pip install langchain langchain-openai


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [56]:
import dotenv
dotenv.load_dotenv()

True

In [59]:
import os
import json
from typing import Dict, Any
import requests
from langchain.tools import BaseTool
from langchain_core.utils.function_calling import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class AmadeusFlightOffersTool(BaseTool):
    """Tool to retrieve flight offers from Amadeus API."""
    
    name: str = "amadeus_flight_offers"
    description: str = "Retrieves flight offers from the Amadeus API."
    
    @classmethod
    def extract_parameters_with_llm(
        cls, 
        query: str, 
        function_spec: Dict[str, Any]
    ) -> Dict[str, Any]:
        """
        Extract API parameters using LLM based on function specification.
        """
        # Prepare a detailed description of parameters
        parameters_details = "\n\t".join([
            f"- {param}: {cls._get_parameter_description(param)} "
            f"(Type: {details.get('type', 'unknown')}, "
            f"Required: {'Yes' if param in function_spec['parameters'].get('required', []) else 'No'})"
            for param, details in function_spec['parameters']['properties'].items()
        ])

        # Create the system prompt
        system_prompt = f"""
        You are an expert at extracting structured parameters for an API flight search function.

        API Function Specification Details:
        Name: {function_spec.get('name', 'Unknown')}
        Description: {function_spec.get('description', 'No description')}

        Parameters:
        {parameters_details}

        Extraction Guidelines:
        1. Carefully analyze the user query to extract values for each parameter
        2. Match parameters exactly as specified in the function specification
        3. Use exact IATA codes for locations
        4. Use YYYY-MM-DD format for dates
        5. By default, assume 1 adult traveler and a maximum of 5 flight offers
        
        Output Instructions:
        - Return a valid JSON object with extracted parameters
        - Only include parameters you can confidently extract
        - Ensure type compatibility
        - If unsure about a parameter, do not include it
        """

        print(f"System Prompt:\n{system_prompt}")

        # Create the prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{query}")
        ])

        # Initialize the LLM
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
        
        # Create the chain
        chain = prompt | llm
        
        # Generate the response
        response = chain.invoke({
            "query": query
        })
        
        # Extract and parse the parameters
        try:
            # Try to parse the LLM response as JSON
            extracted_params = json.loads(response.content)
            
            # Validate required parameters
            required_params = function_spec['parameters'].get('required', [])
            for param in required_params:
                if param not in extracted_params:
                    raise ValueError(f"Missing required parameter: {param}")
            
            return extracted_params
        
        except json.JSONDecodeError:
            # Fallback to minimal defaults
            # TODO RATTAN - Add more sophisticated fallbacks
            return {
                "adults": 1,
                "max": 5
            }
    
    @staticmethod
    def _get_parameter_description(param: str) -> str:
        """
        Provide detailed descriptions for each parameter.
        """
        descriptions = {
            'originLocationCode': 'City/airport IATA code from which the traveler will depart (e.g., BOS for Boston)',
            'destinationLocationCode': 'City/airport IATA code to which the traveler is going (e.g., PAR for Paris)',
            'departureDate': 'Date of departure in ISO 8601 YYYY-MM-DD format (e.g., 2017-12-25)',
            'adults': 'Number of adult travelers (age 12 or older)',
            'max': 'Maximum number of flight offers to return (must be >= 1, default 250)'
        }
        return descriptions.get(param, 'No description available')
    
    def _run(
        self, 
        originLocationCode: str, 
        destinationLocationCode: str, 
        departureDate: str, 
        adults: int, 
        max: int = 5
    ) -> Dict[str, Any]:
        """
        Execute the API call to retrieve flight offers.
        """
        # Retrieve API credentials from environment variables
        client_id = os.getenv('AMADEUS_CLIENT_ID')
        client_secret = os.getenv('AMADEUS_CLIENT_SECRET')
        
        if not client_id or not client_secret:
            raise ValueError("Amadeus API credentials not found. Set AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET.")
        
        # First, get an access token
        token_url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
        token_data = {
            'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret
        }
        
        try:
            # Get access token
            token_response = requests.post(token_url, data=token_data)
            token_response.raise_for_status()
            access_token = token_response.json()['access_token']
            
            # Prepare API call
            api_url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
            headers = {
                'Authorization': f'Bearer {access_token}'
            }
            
            params = {
                'originLocationCode': originLocationCode,
                'destinationLocationCode': destinationLocationCode,
                'departureDate': departureDate,
                'adults': adults,
                'max': max
            }
            
            # Make the API call
            response = requests.get(api_url, headers=headers, params=params)
            response.raise_for_status()
            
            results = response.json()

            if "data" not in results or len(results["data"]) == 0:
                print("No flight offers found.")
                return {"error": "No flight offers found."}

            flightOfferData = results["data"][0]  # taking the first flight offer

            pricing_url = "https://test.api.amadeus.com/v1/shopping/flight-offers/pricing"

            payload = {
                "data": {
                    "type": "flight-offers-pricing",
                    "flightOffers": [
                        flightOfferData
                    ]
                }
            }

            pricing_response = requests.post(pricing_url, headers=headers, json=payload)
            pricing_response.raise_for_status()
            pricing_data = pricing_response.json()
            print("Flight Offer Pricing Results:")
            print(json.dumps(pricing_data, indent=2))

            if "data" not in pricing_data or "flightOffers" not in pricing_data["data"] or len(pricing_data["data"]["flightOffers"]) == 0:
                print("No priced flight offers found.")
                return {"error": "No priced flight offers found."}
            
            flightOfferPriceData = pricing_data["data"]["flightOffers"][0]

            print(f"Flight Offer Price Data: {json.dumps(flightOfferPriceData, indent=2)}")
            
            orders_url = "https://test.api.amadeus.com/v1/booking/flight-orders"

            # flightOfferPriceData1 = {"type":"flight-offer","id":"1","source":"GDS","instantTicketingRequired":False,"nonHomogeneous":False,"paymentCardRequired":False,"lastTicketingDate":"2024-12-19","itineraries":[{"segments":[{"departure":{"iataCode":"CDG","at":"2024-12-19T10:00:00"},"arrival":{"iataCode":"FRA","at":"2024-12-19T14:30:00"},"carrierCode":"6X","number":"501","aircraft":{"code":"744"},"operating":{"carrierCode":"6X"},"duration":"PT4H30M","id":"5","numberOfStops":0,"co2Emissions":[{"weight":71,"weightUnit":"KG","cabin":"ECONOMY"}]},{"departure":{"iataCode":"FRA","at":"2024-12-19T18:10:00"},"arrival":{"iataCode":"ICN","at":"2024-12-20T11:25:00"},"carrierCode":"6X","number":"9744","aircraft":{"code":"744"},"operating":{"carrierCode":"6X"},"duration":"PT9H15M","id":"6","numberOfStops":0,"co2Emissions":[{"weight":404,"weightUnit":"KG","cabin":"ECONOMY"}]}]}],"price":{"currency":"EUR","total":"270.36","base":"134.00","fees":[{"amount":"0.00","type":"SUPPLIER"},{"amount":"0.00","type":"TICKETING"},{"amount":"0.00","type":"FORM_OF_PAYMENT"}],"grandTotal":"270.36","billingCurrency":"EUR"},"pricingOptions":{"fareType":["PUBLISHED"],"includedCheckedBagsOnly":True},"validatingAirlineCodes":["6X"],"travelerPricings":[{"travelerId":"1","fareOption":"STANDARD","travelerType":"ADULT","price":{"currency":"EUR","total":"135.18","base":"67.00","taxes":[{"amount":"4.51","code":"IZ"},{"amount":"3.00","code":"O4"},{"amount":"13.13","code":"QX"},{"amount":"21.89","code":"FR"},{"amount":"25.65","code":"RA"}],"refundableTaxes":"68.18"},"fareDetailsBySegment":[{"segmentId":"5","cabin":"ECONOMY","fareBasis":"YCNV1","class":"Y","includedCheckedBags":{"quantity":9}},{"segmentId":"6","cabin":"ECONOMY","fareBasis":"YCNV1","class":"Y","includedCheckedBags":{"quantity":9}}]},{"travelerId":"2","fareOption":"STANDARD","travelerType":"ADULT","price":{"currency":"EUR","total":"135.18","base":"67.00","taxes":[{"amount":"4.51","code":"IZ"},{"amount":"3.00","code":"O4"},{"amount":"13.13","code":"QX"},{"amount":"21.89","code":"FR"},{"amount":"25.65","code":"RA"}],"refundableTaxes":"68.18"},"fareDetailsBySegment":[{"segmentId":"5","cabin":"ECONOMY","fareBasis":"YCNV1","class":"Y","includedCheckedBags":{"quantity":9}},{"segmentId":"6","cabin":"ECONOMY","fareBasis":"YCNV1","class":"Y","includedCheckedBags":{"quantity":9}}]}]}

            payload = {
                "data": {
                    "type": "flight-order",
                    "flightOffers": [
                        flightOfferPriceData
                    ],
                    "travelers": [
                    {
                        "id": "1",
                        "dateOfBirth": "1982-01-16",
                        "name": {
                            "firstName": "JORGE",
                            "lastName": "GONZALES"
                        },
                        "gender": "MALE",
                        "contact": {
                            "emailAddress": "jorge.gonzales833@telefonica.es",
                            "phones": [
                                {
                                "deviceType": "MOBILE",
                                "countryCallingCode": "34",
                                "number": "480080076"
                                }
                            ]
                        },
                        # "documents": [
                        # {
                        #     "documentType": "PASSPORT",
                        #     "birthPlace": "Madrid",
                        #     "issuanceLocation": "Madrid",
                        #     "issuanceDate": "2015-04-14",
                        #     "number": "00000000",
                        #     "expiryDate": "2025-04-14",
                        #     "issuanceCountry": "ES",
                        #     "validityCountry": "ES",
                        #     "nationality": "ES",
                        #     "holder": True
                        # }
                        # ]
                    },
                    # {
                    #     "id": "2",
                    #     "dateOfBirth": "2012-10-11",
                    #     "gender": "FEMALE",
                    #     "contact": {
                    #     "emailAddress": "jorge.gonzales833@telefonica.es",
                    #     "phones": [
                    #         {
                    #         "deviceType": "MOBILE",
                    #         "countryCallingCode": "34",
                    #         "number": "480080076"
                    #         }
                    #     ]
                    #     },
                    #     "name": {
                    #     "firstName": "ADRIANA",
                    #     "lastName": "GONZALES"
                    #     }
                    # }
                    ],
                    # "remarks": {
                    #     "general": [
                    #         {
                    #         "subType": "GENERAL_MISCELLANEOUS",
                    #         "text": "ONLINE BOOKING FROM INCREIBLE VIAJES"
                    #         }
                    #     ]
                    # },
                    # "ticketingAgreement": {
                    # "option": "DELAY_TO_CANCEL",
                    # "delay": "6D"
                    # },
                    # "contacts": [
                    #     {
                    #         "addresseeName": {
                    #         "firstName": "PABLO",
                    #         "lastName": "RODRIGUEZ"
                    #         },
                    #         "companyName": "INCREIBLE VIAJES",
                    #         "purpose": "STANDARD",
                    #         "phones": [
                    #         {
                    #             "deviceType": "LANDLINE",
                    #             "countryCallingCode": "34",
                    #             "number": "480080071"
                    #         },
                    #         {
                    #             "deviceType": "MOBILE",
                    #             "countryCallingCode": "33",
                    #             "number": "480080072"
                    #         }
                    #         ],
                    #         "emailAddress": "support@increibleviajes.es",
                    #         "address": {
                    #         "lines": [
                    #             "Calle Prado, 16"
                    #         ],
                    #         "postalCode": "28014",
                    #         "cityName": "Madrid",
                    #         "countryCode": "ES"
                    #         }
                    #     }
                    # ]
                }
            }

            print(f"Flight Order Payload: {json.dumps(payload, indent=2)}")

            order_response = requests.post(orders_url, headers=headers, json=payload)
            order_response.raise_for_status()
            order_data = order_response.json()

            print("Flight Order Results:")
            print(json.dumps(order_data, indent=2))

            return order_data
        
        except requests.RequestException as e:
            return {"error": str(e)}
    
    def _arun(self, *args, **kwargs):
        """
        Async run method - not implemented for this tool.
        """
        raise NotImplementedError("Async run is not supported for this tool.")

In [60]:
flight_tool = AmadeusFlightOffersTool()
    
openai_function = format_tool_to_openai_function(flight_tool)
    
query = "Book me a flight for Rattandeep Singh born on 7th March 1998 from New York to Delhi on 30th December 2024?. Contact Number: 8447151548"

print(f"Open AI Function: {json.dumps(openai_function, indent=2)}")

params = AmadeusFlightOffersTool.extract_parameters_with_llm(
    query, 
    openai_function
)
    
print("Extracted Parameters:")
print(json.dumps(params, indent=2))
    
try:
    results = flight_tool._run(**params)
    print("\nFlight Create Order Output:")
    print(json.dumps(results, indent=2))
except Exception as e:
    print(f"Error: {e}")

Open AI Function: {
  "name": "amadeus_flight_offers",
  "description": "Retrieves flight offers from the Amadeus API.",
  "parameters": {
    "properties": {
      "originLocationCode": {
        "type": "string"
      },
      "destinationLocationCode": {
        "type": "string"
      },
      "departureDate": {
        "type": "string"
      },
      "adults": {
        "type": "integer"
      },
      "max": {
        "default": 5,
        "type": "integer"
      }
    },
    "required": [
      "originLocationCode",
      "destinationLocationCode",
      "departureDate",
      "adults"
    ],
    "type": "object"
  }
}
System Prompt:

        You are an expert at extracting structured parameters for an API flight search function.

        API Function Specification Details:
        Name: amadeus_flight_offers
        Description: Retrieves flight offers from the Amadeus API.

        Parameters:
        - originLocationCode: City/airport IATA code from which the traveler will dep